# 계층형 분류기 학습

본 프로젝트에서는 텍스트 데이터를 2단계에 걸쳐 분류하는 계층형 분류기(Hierarchical Classifier)를 구현합니다. 이 접근 방식은 대분류(그룹)를 먼저 판별한 후, 해당 그룹 내에서 세부 분류(서브레딧)를 수행하여 전체적인 분류 정확도와 효율성을 높이는 것을 목표로 합니다.

## 학습 과정

1.  **데이터 준비**: 
    *   **데이터 확인**: 데이터 형태와 결측값 개수 등을 확인합니다.
    *   **전처리**: 결측값 및 불용어 제거 등의 전처리를 수행합니다.
    *   **벡터화**: 전처리된 텍스트 데이터를 `TfidfVectorizer`를 사용하여 고차원 벡터로 변환합니다. 이를 통해 텍스트의 의미적 특징을 수치적으로 표현합니다.

2.  **상위 분류기 (Top-level Classifier) 학습**:
    *   **목표**: 텍스트가 3개의 그룹('Mystery', 'Dev', 'Culture') 중 어디에 속하는지 분류합니다.
    *   **모델**: 대용량 텍스트 데이터에 효율적인 `SGDClassifier`를 사용합니다.
    *   **최적화**: `RandomizedSearchCV`를 이용해 최적의 하이퍼파라미터를 탐색하여 모델의 일반화 성능을 극대화합니다.

3.  **하위 분류기 (Sub-level Classifiers) 학습**:
    *   **목표**: 각 그룹별로 3개의 서브레딧 중 어느 것인지 세부적으로 분류합니다.
    *   **모델**: 그룹별로 독립된 `SGDClassifier`를 학습시켜, 해당 그룹의 데이터 특성에 더 전문화된 모델을 만듭니다.
    *   **최적화**: 각 하위 분류기 역시 `RandomizedSearchCV`를 통해 개별적으로 최적화합니다.

4.  **모델 저장**: 학습이 완료된 상위 분류기와 3개의 하위 분류기 모델을 각각 파일로 저장한다.

## 1. 데이터 준비

### 서브레딧 목록 (총 20개, 5그룹)

수집 데이터의 다양성을 확보하고 유사 범주별 관계성을 분석하기 위해 아래와 같이 3개 그룹, 총 9개의 서브레딧을 선정했습니다.

그룹 | 대상 서브레딧
-- | --
미스터리 및 특이 현상(Mystery) | r/Thetruthishere, r/Glitch_in_the_Matrix, r/UnresolvedMysteries, r/Paranormal
개발 및 커리어(Dev) | r/learnprogramming, r/cscareerquestions, r/SideProject, r/AskProgramming
문화 및 심층 분석(Culture) | r/TrueFilm, r/booksuggestions, r/TrueGaming, r/LetsTalkMusic
개인 사연 및 조언(Life) | r/relationship_advice, r/AmItheAsshole, r/offmychest, r/Advice
경제 및 생활(Finance) | r/personalfinance, r/investing, r/Frugal, r/financialindependence

In [14]:
FILE_PATH = 'assets/' 
MODEL_PATH = 'models/'

In [15]:
# 서브레딧 및 그룹 정의
SUB_PER_GROUP = 4
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries', 'Paranormal',
    'learnprogramming', 'cscareerquestions', 'SideProject', 'AskProgramming',
    'TrueFilm', 'booksuggestions', 'TrueGaming', 'LetsTalkMusic',
    'relationship_advice', 'AmItheAsshole', 'offmychest', 'Advice',
    'personalfinance', 'investing', 'Frugal', 'financialindependence',
]
GROUP_MAP = {
    'Mystery': SUBREDDITS[0:SUB_PER_GROUP], 
    'Dev': SUBREDDITS[SUB_PER_GROUP:2*SUB_PER_GROUP], 
    'Culture': SUBREDDITS[2*SUB_PER_GROUP:3*SUB_PER_GROUP], 
    'Life': SUBREDDITS[3*SUB_PER_GROUP:4*SUB_PER_GROUP],
    'Finance': SUBREDDITS[4*SUB_PER_GROUP:5*SUB_PER_GROUP],
}
VECTOR_DIMENSION = 5000  # 문서 벡터 차원

In [16]:
import pandas as pd
import joblib

# 데이터 로드
train_df = pd.read_csv(FILE_PATH + 'train_data.csv')
test_df = pd.read_csv(FILE_PATH + 'test_data.csv')

subreddit_df = train_df[train_df['subreddit'] == SUBREDDITS[0]].copy()
subreddit_df.head(5)

# 벡터라이저 로드
vectorizer = joblib.load(MODEL_PATH + 'tfidf_vectorizer.pkl')

## 2. 상위 분류기 학습

In [17]:
# 상위 그룹 분류기 학습
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import SGDClassifier
from scipy.stats import loguniform

X_train = vectorizer.transform(train_df['preprocessed_content']).toarray()

X_test = vectorizer.transform(test_df['preprocessed_content']).toarray()

# 라벨링 (상위 그룹)
reverse_group_map = {subreddit: group for group, subreddits in GROUP_MAP.items() for subreddit in subreddits}

# 학습 및 테스트 라벨 생성
y_train = train_df['subreddit'].map(reverse_group_map)
y_test = test_df['subreddit'].map(reverse_group_map)

print(f"데이터 분할 완료: 학습용 {X_train.shape[0]}개, 테스트용 {X_test.shape[0]}개")

# RandomizedSearchCV 설정

# 파라미터 분포 정의
param_dist = {
    'loss': ['log_loss'],  # 로지스틱 회귀 (확률 출력용)
    'alpha': loguniform(1e-5, 1e-1), 
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000, 3000]
}

# 기본 모델 설정
base_model = SGDClassifier(random_state=42, class_weight='balanced', n_jobs=1)

# 교차 검증 전략 (학습 데이터 안에서만 수행)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# RandomizedSearchCV 객체 생성
random_search_top = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=20,          
    cv=cv_strategy,     
    scoring='accuracy', 
    n_jobs=-1,          
    verbose=1,
    random_state=42
)

print("상위 분류기(Top-level Classifier) 최적화 시작 (RandomizedSearch)...")

random_search_top.fit(X_train, y_train)

# 결과 출력 및 최종 평가
print("\n" + "="*50)
print(f"최적의 하이퍼파라미터: {random_search_top.best_params_}")
print(f"교차 검증 최고 점수 (Best CV Score): {random_search_top.best_score_:.4f}")

# 최적의 모델 추출
top_classifier = random_search_top.best_estimator_

# 성능 평가
y_pred = top_classifier.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)

print("-" * 50)
print(f"최종 테스트 정확도 (Test Accuracy): {final_accuracy:.4f}")
print("="*50)
print("\n[상세 분류 리포트]")
print(classification_report(y_test, y_pred))

데이터 분할 완료: 학습용 15736개, 테스트용 3934개
상위 분류기(Top-level Classifier) 최적화 시작 (RandomizedSearch)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits

최적의 하이퍼파라미터: {'alpha': np.float64(1.5339162591163613e-05), 'loss': 'log_loss', 'max_iter': 3000, 'penalty': 'elasticnet'}
교차 검증 최고 점수 (Best CV Score): 0.9512
--------------------------------------------------
최종 테스트 정확도 (Test Accuracy): 0.9441

[상세 분류 리포트]
              precision    recall  f1-score   support

     Culture       0.97      0.95      0.96       798
         Dev       0.93      0.97      0.95       780
     Finance       0.97      0.92      0.94       777
        Life       0.89      0.94      0.92       791
     Mystery       0.97      0.95      0.96       788

    accuracy                           0.94      3934
   macro avg       0.95      0.94      0.94      3934
weighted avg       0.95      0.94      0.94      3934



## 3. 하위 분류기 학습

In [ ]:
# 세부 레딧 분류기 학습
# 그룹별로 별도의 분류기를 저장할 딕셔너리
sub_classifiers = {}
sub_best_params = {}

# 최적화 탐색 범위 설정 (RandomizedSearch용)
param_dist = {
    'loss': ['log_loss'], 
    'alpha': loguniform(1e-5, 1e-1), # 0.00001 ~ 0.1 사이 랜덤 탐색
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000]
}

print("하위 분류기 최적화 및 학습 시작...\n")

y_pred_sub = [] # 하위 분류기 예측값 저장 리스트
for group_name, sub_list in GROUP_MAP.items():
    print(f"--- [{group_name}] 그룹 전용 분류기 최적화 중 ---")
    
    # 해당 그룹 데이터 마스킹 (Train / Test)
    # 이미 로드된 train_df, test_df에서 해당 그룹에 속하는 행만 True
    train_mask = train_df['subreddit'].isin(sub_list)
    test_mask = test_df['subreddit'].isin(sub_list)
    
    # 데이터 필터링 (벡터 및 라벨)
    X_sub_train = X_train[train_mask]
    y_sub_train = train_df.loc[train_mask, 'subreddit']
    
    X_sub_test = X_test[test_mask]
    y_sub_test = test_df.loc[test_mask, 'subreddit']
    
    # RandomizedSearchCV 설정
    base_model = SGDClassifier(random_state=42, class_weight='balanced', n_jobs=1)
    
    search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_dist,
        n_iter=10,          # 그룹당 10번 탐색
        cv=3,               # 그룹별 3-Fold 사용
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    # 최적화 실행
    search.fit(X_sub_train, y_sub_train)
    
    # 최적 모델 저장 및 평가
    best_model = search.best_estimator_
    sub_classifiers[group_name] = best_model
    sub_best_params[group_name] = search.best_params_
    
    # 검증 데이터로 성능 확인
    y_sub_pred = best_model.predict(X_sub_test)
    y_pred_sub.append(y_sub_pred[0])
    acc = accuracy_score(y_sub_test, y_sub_pred)
    
    print(f"  -> 최적 설정: alpha={search.best_params_['alpha']:.5f}, penalty={search.best_params_['penalty']}")
    print(f"  -> 테스트 정확도: {acc:.4f}")

print("\n 모든 하위 분류기 학습 완료.")

하위 분류기 최적화 및 학습 시작...

--- [Mystery] 그룹 전용 분류기 최적화 중 ---
  -> 최적 설정: alpha=0.00007, penalty=l2
  -> 테스트 정확도: 0.7931
--- [Dev] 그룹 전용 분류기 최적화 중 ---
  -> 최적 설정: alpha=0.00031, penalty=elasticnet
  -> 테스트 정확도: 0.7064
--- [Culture] 그룹 전용 분류기 최적화 중 ---
  -> 최적 설정: alpha=0.00007, penalty=l2
  -> 테스트 정확도: 0.9837
--- [Life] 그룹 전용 분류기 최적화 중 ---


## 4. 모델 저장

In [ ]:
# 상위 분류기 모델 저장
best_top_classifier = random_search_top.best_estimator_
joblib.dump(best_top_classifier, MODEL_PATH + 'top_classifier.pkl')

# 하위 분류기 모델 저장
for group_name, model in sub_classifiers.items():
    joblib.dump(model, MODEL_PATH + f'sub_classifier_{group_name}.pkl')